In [1]:
import os
import time
import logging
import yaml
import ast
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from pprgo import utils
from pprgo import ppr
from pprgo import pprgo

import pickle

In [2]:
# Set up logging
logger = logging.getLogger()
logger.handlers = []
ch = logging.StreamHandler()
formatter = logging.Formatter(
        fmt='%(asctime)s (%(levelname)s): %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')
ch.setFormatter(formatter)
logger.addHandler(ch)
logger.setLevel('INFO')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
# !wget --show-progress -O data/reddit.npz https://ndownloader.figshare.com/files/23742119  #Reddit
# !wget --show-progress -O data/magc.npz https://figshare.com/ndownloader/files/24045741 #MAG-COARSE

# Download dataset

# Load config

In [4]:
with open('config_demo.yaml', 'r') as c:
    config = yaml.safe_load(c)

In [5]:
# For strings that yaml doesn't parse (e.g. None)
for key, val in config.items():
    if type(val) is str:
        try:
            config[key] = ast.literal_eval(val)
        except (ValueError, SyntaxError):
            pass

In [6]:
data_file           = config['data_file']           # Path to the .npz data file
# data_file = 'data/cora_full.npz'
# data_file = 'data/pubmed.npz'
data_file = 'data/reddit.npz'
# data_file = 'data/magc.npz'

split_seed          = config['split_seed']          # Seed for splitting the dataset into train/val/test
ntrain_div_classes  = config['ntrain_div_classes']  # Number of training nodes divided by number of classes
attr_normalization  = config['attr_normalization']  # Attribute normalization. Not used in the paper

alpha               = config['alpha']               # PPR teleport probability
alpha = 0.50
eps                 = config['eps']                 # Stopping threshold for ACL's ApproximatePR
topk                = config['topk']                # Number of PPR neighbors for each node
topk=32
ppr_normalization   = config['ppr_normalization']   # Adjacency matrix normalization for weighting neighbors

hidden_size         = config['hidden_size']         # Size of the MLP's hidden layer
nlayers             = config['nlayers']             # Number of MLP layers
weight_decay        = config['weight_decay']        # Weight decay used for training the MLP
dropout             = config['dropout']             # Dropout used for training

lr                  = config['lr']                  # Learning rate
max_epochs          = config['max_epochs']          # Maximum number of epochs (exact number if no early stopping)
batch_size          = config['batch_size']          # Batch size for training
batch_mult_val      = config['batch_mult_val']      # Multiplier for validation batch size

eval_step           = config['eval_step']           # Accuracy is evaluated after every this number of steps
run_val             = config['run_val']             # Evaluate accuracy on validation set during training

early_stop          = config['early_stop']          # Use early stopping
patience            = config['patience']            # Patience for early stopping

nprop_inference     = config['nprop_inference']     # Number of propagation steps during inference
inf_fraction        = config['inf_fraction']        # Fraction of nodes for which local predictions are computed during inference

In [7]:
# import warnings
# warnings.filterwarnings("ignore")

# Load the data

In [8]:
start = time.time()
(adj_matrix, attr_matrix, labels,
 train_idx, val_idx, test_idx) = utils.get_data(
        f"{data_file}",
        seed=split_seed,
        ntrain_div_classes=ntrain_div_classes,
        normalize_attr=attr_normalization
)
try:
    d = attr_matrix.n_columns
except AttributeError:
    d = attr_matrix.shape[1]
nc = labels.max() + 1
print(nc)

# print('shape attibute matrix: ', attr_matrix.n_rows)
# print('Training: ', len(train_idx))
# print('Validation: ', len(val_idx))
# print('Testing: ', len(test_idx))
# print('train_idx: ', train_idx)

time_loading = time.time() - start
print(f"Runtime: {time_loading:.2f}s")

41
Runtime: 18.62s


# Preprocessing: Calculate PPR scores

In [9]:
#Compute core numbers
# core_numbers = np.load('core-numbers-networkx.npy')

# graph = igraph.Graph.Adjacency((adj_matrix.todense()> 0).tolist())
# core_numbers = np.array(graph.coreness())
# try:
#     core_numbers = np.load('coredata/reddit-cores.npy')
#     print('Cores loaded ')
# except:
#     print('No file for core numbers')
#     core_numbers = None

# core_numbers = None
core_numbers = np.load('coredata/reddit-cores.npy')

In [10]:
# with open('graph.pkl', 'wb') as outp:  # Overwrites any existing file.
#         pickle.dump(graph, outp, pickle.HIGHEST_PROTOCOL)

# with open('graph.pkl', 'rb') as inp:
#     graph = pickle.load(inp)

# shortest_path = graph.get_shortest_paths(0, to=[1,2,3])
# print(shortest_path)

In [11]:
# compute the ppr vectors for train/val nodes using ACL's ApproximatePR

start = time.time()

# topk_train = ppr.topk_ppr_matrix(adj_matrix, alpha, eps, train_idx, topk, #core_numbers, g_networkX,
#                                  normalization=ppr_normalization) #, S=1, gamma=0.8)

topk_train, core_topk_train, coreRank, mean_kn = ppr.topk_ppr_matrix(adj_matrix, alpha, eps, train_idx, topk, core_numbers,
                                 normalization=ppr_normalization)
if run_val:
    topk_val = ppr.topk_ppr_matrix(adj_matrix, alpha, eps, val_idx, topk,
                                   normalization=ppr_normalization)
else:
    topk_val = None

time_preprocessing = time.time() - start
print(f"Runtime: {time_preprocessing:.2f}s")


# print('topk_train', topk_train.shape)
# adj_xd = adj_matrix[train_idx]
# print(adj_xd[0:0 + batch_size].shape)

Runtime: 7.23s


In [12]:
import scipy.sparse

# scipy.sparse.save_npz('FwdPush-topk_train.npz', topk_train)

In [13]:
# topk_train = scipy.sparse.load_npz('FwdPush-topk_train.npz')

In [14]:
print(topk_train.shape)
# print(core_topk_train)

(820, 232965)


# Training: Set up model and train

In [15]:
start = time.time()
tf.reset_default_graph()
tf.set_random_seed(0)

K = 5

#ppr like
TEMP = alpha*(1-alpha)**np.arange(K+1)
TEMP[-1] = (1-alpha)**K
TEMP = TEMP.astype('f')


#negative ppr 
# TEMP = (alpha)**np.arange(K+1)
# TEMP = TEMP/np.sum(np.abs(TEMP))
# TEMP = TEMP.astype('f')

#Random
# bound = np.sqrt(3/(K+1))
# TEMP = np.random.uniform(-bound, bound, K+1)
# TEMP = TEMP/np.sum(np.abs(TEMP))
# TEMP = TEMP.astype('f')


# print(TEMP)
# print(TEMP.dtype)

#random version
gamma = 0.5
# gamma = np.array([0.8, 0.2])
# gamma = gamma.astype('f')

model = pprgo.PPRGo(d, nc, hidden_size, nlayers, lr, gamma, weight_decay, dropout, #adj_matrix, train_idx, TEMP, K, topk_train, intermediate_layer=100,
                    sparse_features=type(attr_matrix) is not np.ndarray)



# print(type(attr_matrix) is not np.ndarray)

2022-08-08 15:44:59 (WARNING): From /var/folders/x5/bhlqr68144x_138bllx2mdwh0000gn/T/ipykernel_49287/3408356976.py:2: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.



2022-08-08 15:44:59 (WARNING): From /var/folders/x5/bhlqr68144x_138bllx2mdwh0000gn/T/ipykernel_49287/3408356976.py:3: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.



In [16]:
sess = tf.compat.v1.Session()
with sess.as_default():
    tf.compat.v1.global_variables_initializer().run()
    nepochs, loss_hist, acc_hist, f1_hist = pprgo.train(
            sess=sess, model=model, attr_matrix=attr_matrix,
            train_idx=train_idx, val_idx=val_idx,
            topk_train=topk_train, topk_val=topk_val, core_topk_train=core_topk_train,
            labels=labels, #adj_matrix=adj_matrix,
            max_epochs=max_epochs, batch_size=batch_size, batch_mult_val=batch_mult_val,
            eval_step=eval_step, early_stop=early_stop, patience=patience)
time_training = time.time() - start
logging.info('Training done.')
print(f"Runtime: {time_training:.2f}s")

2022-08-08 15:45:01 (INFO): Epoch 9, step 20: train 3.72448
2022-08-08 15:45:01 (INFO): Epoch 19, step 40: train 3.33340
2022-08-08 15:45:02 (INFO): Epoch 29, step 60: train 3.16890
2022-08-08 15:45:02 (INFO): Epoch 39, step 80: train 3.02512
2022-08-08 15:45:02 (INFO): Epoch 49, step 100: train 2.94958
2022-08-08 15:45:03 (INFO): Epoch 59, step 120: train 2.85968
2022-08-08 15:45:03 (INFO): Epoch 69, step 140: train 2.69952
2022-08-08 15:45:03 (INFO): Epoch 79, step 160: train 2.66638
2022-08-08 15:45:03 (INFO): Epoch 89, step 180: train 2.51879
2022-08-08 15:45:03 (INFO): Epoch 99, step 200: train 2.50320
2022-08-08 15:45:04 (INFO): Epoch 109, step 220: train 2.40928
2022-08-08 15:45:04 (INFO): Epoch 119, step 240: train 2.28336
2022-08-08 15:45:04 (INFO): Epoch 129, step 260: train 2.27942
2022-08-08 15:45:04 (INFO): Epoch 139, step 280: train 2.21014
2022-08-08 15:45:05 (INFO): Epoch 149, step 300: train 2.19395
2022-08-08 15:45:05 (INFO): Epoch 159, step 320: train 2.11384
2022-08

Runtime: 6.28s


In [17]:
# print(model.batch_labels)
# print(model.adjacency_weighted_logits)

# Inference (val and test)

In [18]:
start = time.time()
predictions, time_logits, time_propagation = model.predict(
        sess=sess, adj_matrix=adj_matrix, attr_matrix=attr_matrix, alpha=alpha, coreRank=coreRank,
        nprop=nprop_inference, inf_fraction=inf_fraction,
        ppr_normalization=ppr_normalization)
time_inference = time.time() - start
print(f"Runtime: {time_inference:.2f}s")

Inference gamma:  0.14363955
Runtime: 27.98s


In [19]:
# deg = adj_matrix.sum(1).A1
# deg_sqrt_inv = 1. / np.sqrt(np.maximum(deg, 1e-12))

# core_matrix = adj_matrix.multiply(coreRank)
# normalized_core_matrix = core_matrix.multiply(1/core_matrix.sum(axis=1).A1[:, None])
# print(normalized_core_matrix.sum(axis=1))







In [20]:
# xd =  deg_sqrt_inv[:, None] *(normalized_core_matrix * deg_sqrt_inv[:, None])
# print(xd.shape)
# adj = np.array([[0,1,1],[1,1,0], [1,0,1]])
# coreRank = np.array([3,7,9])

# print('adj: ', adj)
# print('coreRank: ', coreRank)

# coreMatrix = np.multiply(adj, coreRank)

# print('coreMatrix: ', coreMatrix)

# print('sum: ', np.sum(coreMatrix, axis=1))

# z = np.multiply(coreMatrix, 1/ np.sum(coreMatrix, axis=1)[:,None])
# print(z)

# print(np.sum(z, axis=1))


In [21]:
print(acc_hist)


{'train': [0.14935064935064934, 0.1525974025974026, 0.17532467532467533, 0.21428571428571427, 0.2532467532467532, 0.2564935064935065, 0.288961038961039, 0.33766233766233766, 0.3538961038961039, 0.35714285714285715, 0.3961038961038961, 0.40584415584415584, 0.42207792207792205, 0.4253246753246753, 0.44805194805194803, 0.487012987012987, 0.4642857142857143, 0.5032467532467533, 0.512987012987013, 0.551948051948052], 'val': []}


# Collect and print results

In [22]:

acc_train = 100 * accuracy_score(labels[train_idx], predictions[train_idx])
acc_val = 100 * accuracy_score(labels[val_idx], predictions[val_idx])
acc_test = 100 * accuracy_score(labels[test_idx], predictions[test_idx])

f1_train = f1_score(labels[train_idx], predictions[train_idx], average='macro')
f1_val = f1_score(labels[val_idx], predictions[val_idx], average='macro')
f1_test = f1_score(labels[test_idx], predictions[test_idx], average='macro')



#gpu_max_bytes = tf.contrib.memory_stats.MaxBytesInUse()
#gpu_memory = sess.run(gpu_max_bytes)
memory = utils.get_max_memory_bytes()

time_total = time_preprocessing + time_training + time_inference

In [23]:
print(f'''
Accuracy: Train: {acc_train:.1f}%, val: {acc_val:.1f}%, test: {acc_test:.1f}%
F1 score: Train: {f1_train:.3f}, val: {f1_val:.3f}, test: {f1_test:.3f}

Runtime: Preprocessing: {time_preprocessing:.2f}s, training: {time_training:.2f}s, inference: {time_inference:.2f}s -> total: {time_total:.2f}s
Memory: Main: {(memory/1024) / 2**30:.2f}GB
''')

#Memory: Main: {memory / 2**30:.2f}GB, GPU: {gpu_memory / 2**30:.3f}GB


Accuracy: Train: 43.5%, val: 28.7%, test: 28.9%
F1 score: Train: 0.255, val: 0.131, test: 0.134

Runtime: Preprocessing: 7.23s, training: 6.28s, inference: 27.98s -> total: 41.49s
Memory: Main: 2.87GB

